<a href="https://colab.research.google.com/github/A-K-ay/torrent_to_mega/blob/main/torrent_to_mega_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Mega

### Add Required User Inputs

In [ ]:
# Mega Credentials

megaEmail = "" #Email/Username
megaPassword = ""    #Password

# Torrent Magnet Link
torrentMagentLink = ""


### Import Dependencies


In [ ]:
!apt install python3-libtorrent

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libtorrent-rasterbar9
Suggested packages:
  libtorrent-rasterbar-dbg
The following NEW packages will be installed:
  libtorrent-rasterbar9 python3-libtorrent
0 upgraded, 2 newly installed, 0 to remove and 23 not upgraded.
Need to get 1,704 kB of archives.
After this operation, 6,479 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libtorrent-rasterbar9 amd64 1.1.13-1.1build2 [1,368 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 python3-libtorrent amd64 1.1.13-1.1build2 [336 kB]
Fetched 1,704 kB in 2s (1,049 kB/s)
Selecting previously unselected package libtorrent-rasterbar9.
(Reading database ... 129506 files and directories currently installed.)
Preparing to unpack .../libtorrent-rasterbar9_1.1.13-1.1build2_amd64.deb ...
Unpacking libtorrent-rasterbar9 (1.1.13-1.1buil

In [ ]:
!pip install mega.py

### Code to download torrent

In [ ]:
import libtorrent as lt
import time
import datetime

ses = lt.session()
ses.listen_on(6881, 6891)
params = {
    'save_path': '/torrents',
    'storage_mode': lt.storage_mode_t(2),
    'paused': False,
    'auto_managed': True,
    'duplicate_is_error': True}
print(torrentMagentLink)

handle = lt.add_magnet_uri(ses, torrentMagentLink, params)
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())

print ('Downloading Metadata...')
while (not handle.has_metadata()):
    time.sleep(1)
print ('Got Metadata, Starting Torrent Download...')

torrentFolderName = handle.name()
print("Starting", handle.name())

while (handle.status().state != lt.torrent_status.seeding):
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata', \
            'downloading', 'finished', 'seeding', 'allocating']
    print ('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
            (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
            s.num_peers, state_str[s.state]))
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")

print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")

print(datetime.datetime.now())

https://yts.lt/torrent/download/866BDCFA006930A718ADBC21D8CAE1F2C7F9D8B3
2023-01-25 08:02:28.164601
Got Metadata, Starting Torrent Download...
Starting Avengers Infinity War (2018) [BluRay] [1080p] [YTS.AM]
0.00% complete (down: 0.0 kb/s up: 0.0 kB/s peers: 0) downloading 
0.13% complete (down: 753.2 kb/s up: 25.5 kB/s peers: 14) downloading 
0.79% complete (down: 2760.5 kb/s up: 91.4 kB/s peers: 21) downloading 
1.46% complete (down: 3469.0 kb/s up: 113.4 kB/s peers: 17) downloading 
2.04% complete (down: 3207.0 kb/s up: 111.6 kB/s peers: 14) downloading 
2.59% complete (down: 2985.8 kb/s up: 112.9 kB/s peers: 23) downloading 
3.18% complete (down: 3114.8 kb/s up: 118.3 kB/s peers: 76) downloading 
3.60% complete (down: 2705.5 kb/s up: 123.4 kB/s peers: 75) downloading 
4.16% complete (down: 2802.1 kb/s up: 125.6 kB/s peers: 18) downloading 
4.70% complete (down: 2738.4 kb/s up: 118.2 kB/s peers: 15) downloading 
5.41% complete (down: 3524.8 kb/s up: 136.1 kB/s peers: 16) downloading 

### Create a Zip of the downloaded torrent

In [ ]:
import shutil
begin = time.time()
shutil.make_archive(torrentFolderName, 'zip', "/torrents/"+torrentFolderName)
end = time.time()
print("Compression Successful")
print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")


Compression Successful
Elapsed Time:  1 min : 59 sec


### Upload Zip To Mega

In [ ]:
from mega import Mega
mega = Mega()

# Login
# Add your credentials here

m = mega.login(megaEmail,megaPassword)


begin = time.time()

# Upload File
file = m.upload('/content/'+torrentFolderName+".zip")
end = time.time()
print(torrentFolderName, "COMPLETE")

print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")
